# Two Circular Qubits

Этот notebook рисует два несвязанных кубита, подсоединенных к одной TL. Он использует 3 файла, рисующих отдельные элементы дизайна, и класс Sample из этого нотебука, который их клеит в один дизайн gds.
Для работы необходимо установить библиотеку gdspy.
Создание дизайна происходит в следующем порядке:


   *   **Земля и контактные площадки**. По идее, надо задавать только координаты контактных площадок, поворачивать их или нет код должен определять сам (если площадка отстоит более чем на 900 $\mu m$, происходит поворот). За это отвечает функция 
   
   *Generate_Pads (размер образца по х, размер образца по у, [d], [d1], [d2], [слой], [координаты контактных площадок])*. 
   
   
   *   **TL**. Связывает две контактных площадки слева направо. Вообще все, что содержит копланары, в этом коде надо рисовать слева направо.
   
   *Generate_TL (площадка_1, площадка_2, d, d1, d2)*
   
   
   *   **Резонаторы** сразу **вместе с клешнями** "крест" и "круг". Для них задаются координаты нижнего левого угла в формате coordinates(x, y). Резонаторы **не генерируются** в итоговой ячейке cell после выполнения опреации, так как из клешни надо вычесть дополнительно петельку.
   
       *Generate_Resonators (массив начальных точек, [радиус круговой клешни], [gap], [частота резонатора], [режим (up or down)], [слой])*.
       
       
   *   **Джозефсоны**. 
   
   *Generate_JJ* ([координаты центра нижней границы], [a], [b], [c], [d], [h], [w1], [w2], слой, [режимы])
   
   
   *   **Петельки**. На этом этапе в ячейку sample добавятся и резонаторы, и петельки.
   
   *Generate_Loops* ([координаты второго конца; первый автоматически присоединяется к площадке], [режимы], [d], [d1], [d2], [d для площадки], [d1 для площадки], [d2 для площадки], [массив координат поворотов], [массив углов поворотов], слой)
   
   
   *   **Сетка**. Можно генерировать только после того, как сгенерировано все, что хочется.
   
   *Generate_Grid* (размер образца по х, размер образца по у, слой1, слой2)
   
 В случае, если что-то надо поправить, придется, к сожалению, делать restart.

In [1]:
from qsweepy.design import *
from qsweepy.resonator import*
from qsweepy.loops_and_JJs import*

ModuleNotFoundError: No module named 'qsweepy'

In [2]:
class Sample:
    
    #Содержит функции, генерирующие все площадки, TL, все джозефсоны, все петельки
    
    def __init__(self, number):
        self.name = 'sample' + str(number)
        self.cell = gdspy.Cell(self.name)
    
    def Generate_Pads(self, a, b, number, d_arr, d1_arr, d2_arr, layer, coords = []):
        self.Pads = Pads(a, b, number, d_arr, d1_arr, d2_arr, layer, coords)
        self.Pads_cell = gdspy.Cell('Pads')
        self.Pads_cell.add(self.Pads.Generate_Ground())
        self.cell.add(gdspy.CellReference(self.Pads_cell, (0, 0)))
        
    def Generate_TL(self, start_pad, finish_pad, d, d1, d2, layer):
        TL = Transmission_Line(self.Pads, start_pad, finish_pad, d, d1, d2, layer)
        self.tl = TL
        self.tl_cell = gdspy.Cell('TL')
        self.tl_cell.add(TL.Generate(coords = [], rots = [])) 
        self.cell.add([gdspy.CellReference(self.tl_cell, (0, 0))])
      
    def Generate_Resonators(self, start_points, end_points, radiuses, gaps, freqs, modes, layer, circle = True):
        self.resonators = []
        self.res_cells = []
        self.res_references = []
        for i, (start, end, r, w, freq, mode) in enumerate(zip(start_points, end_points, radiuses, gaps, freqs, modes)):
            self.resonators.append(Resonator(start.x, start.y, end, freq, layer, number = i))
            self.res_cells.append(gdspy.Cell('res' + str(i)))
            self.res_cells[i].add(self.resonators[i].Generate(mode, circle, r = r, w = w))
            #self.cell.add(gdspy.CellReference(self.res_cells[i], (0, 0)))
            
    def Generate_JJ(self, corners, r1, r2, r3, r4, r5, r6, r7, layer, modes):
        self.jj = []
        self.jj_cells = []
        for i, (corner, a, b, c, d, h, w1, w2, mode) in enumerate(zip(corners, r1, r2, r3, r4, r5, r6, r7, modes)):
            self.jj.append(JJ(a, b, c, d, h, w1, w2, layer))
            self.jj_cells.append(gdspy.Cell('jj' + str(i)))
            self.jj_cells[i].add(self.jj[i].Generate(corner.x, corner.y, mode))
            self.cell.add(gdspy.CellReference(self.jj_cells[i], (0,0)))#(corner.x, corner.y)))
            
    def Generate_Loops(self, rightc, modes, d, d1, d2, d_tl, d1_tl, d2_tl, coords_arr, rots_arr, layer):#
        self.loops = []
        self.loop_cells = []
        for i, (right, mode, coords, rots) in enumerate(zip(rightc, modes, coords_arr, rots_arr)):#
            self.loops.append(Loop(right.x, right.y, d, d1, d2, d_tl, d1_tl, d2_tl, mode, 
                                   self.Pads.reference_points[i + 2], layer))
            self.loop_cells.append(gdspy.Cell('loop' + str(i)))
            self.loop_cells[i].add(self.loops[i].Generate(coords, rots, mode))
            new_res = gdspy.boolean(gdspy.CellReference(self.res_cells[i], (0, 0)),
                                              self.loops[i].restricted_area, 'not')
            #self.res_cells[i].polygons = []
            #self.res_cells[i].add(new_res)
            #self.cell.add(gdspy.CellReference(self.loop_cells[i], (0,0)))
            
            self.cell.add(gdspy.boolean(gdspy.boolean(self.res_cells[i].get_polygons(),
                                        self.loops[i].restricted_area, 'not'),
                          gdspy.CellReference(self.loop_cells[i], (0,0)), 'or'))
            
    def Generate_Grid(self, a, b, l1, l2):
        self.grid_cell = gdspy.Cell('Grid')
        res1 = gdspy.Rectangle((400, 400), (402, b - 400), layer = l1) 
        x = 447
        while x < a - 400:
            r1 = gdspy.Rectangle((x, 400), (x + 2, b - 400), layer = l1) 
            x += 47
            res1 = gdspy.boolean(res1, r1, 'or')
            #self.grid_cell.add(r1)
        y = 400
        res2 = gdspy.Rectangle((400, 400), (400, 402), layer = l2) 
        while y < b - 400:
            r1 = gdspy.Rectangle((400, y), (a - 400, y + 2), layer = l2) 
            y += 47
            res2 = gdspy.boolean(res2, r1, 'or')
            #self.grid_cell.add(r1)
        res2 = gdspy.boolean(res2, res1, 'not')
        for area in self.Pads.restricted_area:
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        for area in self.tl.restricted_area:
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        for i in range(len(self.resonators)):
            area = self.resonators[i].restricted_area
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        for i in range(len(self.loops)):
            area = self.loops[i].restricted_area
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        #res = gdspy.boolean(res, self.Pads_cell.get_polygonsets(), 'not')
        self.grid_h = res1
        self.grid_v = res2
        self.grid_cell.add(res1)
        self.grid_cell.add(res2)
        self.cell.add(gdspy.CellReference(self.grid_cell, (0, 0)))
    #self.Resonators
    #self.JJ
    #self.Loops
    #self.TL

In [3]:
# I don't think you should change here smth
sample = Sample(2)
sample.Generate_Pads(5000, 5000, #width, depth
                     4, #number of Pads
                     [17, 17, 10, 10], #d
                     [33, 33, 22, 22], #d1
                     [75, 75, 64, 64], #d2
                     0, #layer
                     [coordinates(800, 2500), coordinates(4200, 2500),
                     coordinates(2500, 4200), coordinates(2500, 800)]) #reference coordinates for Pads
sample.Generate_TL(0, 1, 17, 33, 75, 0)

#Start changing from here
sample.Generate_Resonators([coordinates(sample.tl.start.x + 1200, 
                                        sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d),
                           coordinates(sample.tl.start.x + 1000, 
                                        sample.tl.start.y - (sample.tl.d2 - sample.tl.d)/2)], # ref coordinates
                           [sample.tl.start.x + 2100, sample.tl.end.x - 1000], 
                           [87, 200],#[87, 94], #radiuses for circular claws
                           [10, 100],#[10, 15], #gaps for circular claws (*)
                           [8e9, 9e9], #freqs
                           ['up', 'down'], #mode
                           [0, 0], #layers
                           True) #yes, we want circulare claw
sample.Generate_JJ([sample.resonators[0].claw_center, sample.resonators[1].claw_center], 
                   [0.3, 0.3], # a
                   [0.1, 0.1], #b
                   [0.25, 0.25], #c
                   [0.1, 0.1], #d
                   [10, 100], #h - копипасть сюда строку (*)
                   [7, 7], #w1
                   [4, 4], #w2
                   9, #layer
                   ['up', 'down']) #mode

NameError: name 'gdspy' is not defined

In [4]:
#It is better to leave as it is
sample.Generate_Loops([coordinates(sample.jj[0].ref_x - 6, sample.jj[0].ref_y), 
                       coordinates(sample.jj[1].ref_x + 6, sample.jj[1].ref_y)], 
                      ['down', 'up'], 4, 10, 52, 10, 22, 64,
                      [[coordinates(2087, 4165), coordinates(2527, 4264)],
                       [coordinates(2526, 1000), coordinates(3354, 1083)]],
                      [[rotations(np.pi, np.pi/2), rotations(3*np.pi/2, 2*np.pi)],
                       [rotations(np.pi, np.pi/2), rotations(3*np.pi/2, 2*np.pi)]], layer = 0)

NameError: name 'sample' is not defined

In [5]:
sample.Generate_Grid(5000, 5000, 2, 3)
gdspy.LayoutViewer(depth = 2) # it is important to choose depth!!!

NameError: name 'sample' is not defined

In [6]:
gdspy.LayoutViewer(depth = 2)

NameError: name 'gdspy' is not defined

In [7]:
gdspy.write_gds('capacitances_eq_8e-14.gds', unit=1.0e-6, precision=1.0e-9)

NameError: name 'gdspy' is not defined

In [8]:
sample.jj[0].ref_x, sample.jj[0].ref_y

NameError: name 'sample' is not defined

In [9]:
sample.jj[1].ref_x, sample.jj[1].ref_y

NameError: name 'sample' is not defined

In [10]:
#Change positions of Pads!!!

In [4]:
sample.Generate_Loops([coordinates(sample.jj[0].ref_x - 6, sample.jj[0].ref_y + 10)], 
                      ['down'], 4, 10, 52, 10, 22, 64,
                      [[coordinates(2087, 4165), coordinates(2527, 4264)]],
                      [[rotations(np.pi, np.pi/2), rotations(3*np.pi/2, 2*np.pi)]], layer = 0)

<class 'list'> [(2060.2, 4045.4646940230414), (2060.2, 4165), (2527.0, 4264), (2527.0, 4264.0)]
<class 'gdspy.PolygonSet'>
True PolygonSet (1 polygons, 44 vertices, layers [0], datatypes [0])
False PolygonSet (1 polygons, 44 vertices, layers [0], datatypes [0])
True PolygonSet (1 polygons, 44 vertices, layers [0], datatypes [0])
False PolygonSet (1 polygons, 44 vertices, layers [0], datatypes [0])
True PolygonSet (1 polygons, 66 vertices, layers [0], datatypes [0])
False PolygonSet (1 polygons, 66 vertices, layers [0], datatypes [0])
True PolygonSet (1 polygons, 60 vertices, layers [0], datatypes [0])
False PolygonSet (1 polygons, 60 vertices, layers [0], datatypes [0])


In [ ]:
sample.resonators[0].claw_center.x -= 4
sample.resonators[1].claw_center.x -= 4#-8
sample.resonators[0].claw_center.y -= 2
sample.resonators[1].claw_center.y -= 11
sample.Generate_JJ([sample.resonators[0].claw_center, sample.resonators[1].claw_center], [0.25, 0.25], 
                   [0.695, 0.695], [0.1, 0.1], [0.79, 0.79], [2.2, 2.2], 
                   [0.29, 0.29], [2.2, 2.2], [0.1, 0.1], 9)
#sample.Generate_JJ([sample.resonators[1].claw_center], [0.25], [0.695], [0.1], [0.79], [2.2], [0.29], [2.2], [0.1], 1)
sample.Generate_Loops([coordinates(sample.jj[1].ref_x, sample.jj[1].ref_y), coordinates(sample.jj[0].ref_x, sample.jj[0].ref_y)],
                      [coordinates(sample.jj[1].ref_x + 7, sample.jj[1].ref_y + 2), coordinates(sample.jj[0].ref_x - 3, sample.jj[0].ref_y + 10)], 
                      ['up', 'down'], 4, 10, 52, 10, 22, 64,
                      [[coordinates(2526, 1000), coordinates(3354, 1083)], [coordinates(2087, 4165), coordinates(2527, 4264)]],
                      [[rotations(np.pi, np.pi/2), rotations(3*np.pi/2, 2*np.pi)],[rotations(np.pi, np.pi/2), rotations(3*np.pi/2, 2*np.pi)]], layer = 0)
#sample.Generate_Loops([coordinates(2065, 3861.5), coordinates(2065, 3861.5)],
                      #[coordinates(3397, 1487), coordinates(2064, 3861)], ['up', 'down'], 4, 10, 52, 10, 22, 64,
                      #[[coordinates(2526, 1000), coordinates(3354, 1083)], [coordinates(2087, 4165), coordinates(2527, 4264)]],
                      #[[rotations(np.pi, np.pi/2), rotations(3*np.pi/2, 2*np.pi)],[rotations(np.pi, np.pi/2), rotations(3*np.pi/2, 2*np.pi)]], layer = 0)
#ref1 = gdspy.CellReference(sample.Pads_cell, (0, 0))
#sample_cell = gdspy.Cell('Sample')
#sample_cell.add([ref1])

In [3]:
sample = Sample(2)
sample.Generate_Resonators([coordinates(0, 0), coordinates(3000, 0), coordinates(7000, 0), coordinates(11000, 0),
                           coordinates(15000, 0)],
                           [100, 125, 150, 175, 200], 
                           [0.04*100, 10, 0.16*150, 0.21*175, 60])
#for i in range(len(sample.claw_center)):
    #sample.claw_center[i].x -= 4
    #sample.claw_center[i].y -= 2
sample.Generate_JJ([sample.claw_center[0], sample.claw_center[1], sample.claw_center[2], sample.claw_center[3],
                   sample.claw_center[4]], 
                   [0.3, 0.3, 0.3, 0.3, 0.3, 0.3], 
                   [0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
                   [0.25, 0.25, 0.25, 0.25, 0.25, 0.25], 
                   [0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
                   [0.04*100, 10, 0.16*150, 0.21*175, 60], 
                   [7, 7, 7, 7, 7, 7], 
                   [4, 4, 4, 4, 4, 4], 9)
#sample.Generate_JJ([sample.resonators[1].claw_center], [0.25], [0.695], [0.1], [0.79], [2.2], [0.29], [2.2], [0.1], 1)
sample.Generate_Loops([coordinates(sample.jj[0].ref_x - 6, sample.jj[0].ref_y + 0.04*100), 
                       coordinates(sample.jj[1].ref_x - 6, sample.jj[1].ref_y + 10),
                       coordinates(sample.jj[2].ref_x - 6, sample.jj[2].ref_y + 0.16*150), 
                       coordinates(sample.jj[3].ref_x - 6, sample.jj[3].ref_y + 0.21*175),
                       coordinates(sample.jj[4].ref_x - 6, sample.jj[4].ref_y + 60),], 
                      4, 10, 52, 10, 22, 64, 1)
gdspy.LayoutViewer(depth = 2)

<gdspy.viewer.LayoutViewer object .!layoutviewer>

In [4]:
10/0.115, 15/0.16, 20/0.195, 25/0.23, 30/0.26

(86.95652173913044,
 93.75,
 102.56410256410255,
 108.69565217391303,
 115.38461538461539)

In [4]:
gdspy.write_gds('capacitances_eq_8e-14.gds', unit=1.0e-6, precision=1.0e-9)

In [3]:
sample = Sample(2)
sample.Generate_Resonators([coordinates(0, 0), coordinates(3000, 0), coordinates(7000, 0), coordinates(11000, 0),
                           coordinates(0, 3000), coordinates(3000, 3000), coordinates(7000, 3000), coordinates(11000, 3000),
                           coordinates(0, 6000), coordinates(3000, 6000), coordinates(7000, 6000), coordinates(11000, 6000)],
                           [50, 100, 150, 200, 50, 100, 150, 200], 
                           [5, 5, 5, 5, 35, 35, 35, 35])
#for i in range(len(sample.claw_center)):
    #sample.claw_center[i].x -= 4
    #sample.claw_center[i].y -= 2
sample.Generate_JJ([sample.claw_center[0], sample.claw_center[1], sample.claw_center[2], sample.claw_center[3],
                   sample.claw_center[4], sample.claw_center[5], sample.claw_center[6], sample.claw_center[7]], 
                   [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3], 
                   [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
                   [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25], 
                   [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
                   [5, 5, 5, 5, 35, 35, 35, 35], 
                   [7, 7, 7, 7, 7, 7, 7, 7], 
                   [4, 4, 4, 4, 4, 4, 4, 4], 9)
#sample.Generate_JJ([sample.resonators[1].claw_center], [0.25], [0.695], [0.1], [0.79], [2.2], [0.29], [2.2], [0.1], 1)
sample.Generate_Loops([coordinates(sample.jj[0].ref_x - 6, sample.jj[0].ref_y + 5), 
                       coordinates(sample.jj[1].ref_x - 6, sample.jj[1].ref_y + 5),
                       coordinates(sample.jj[2].ref_x - 6, sample.jj[2].ref_y + 5), 
                       coordinates(sample.jj[3].ref_x - 6, sample.jj[3].ref_y + 5),
                       coordinates(sample.jj[4].ref_x - 6, sample.jj[4].ref_y + 35), 
                       coordinates(sample.jj[5].ref_x - 6, sample.jj[5].ref_y + 35),
                       coordinates(sample.jj[6].ref_x - 6, sample.jj[6].ref_y + 35), 
                       coordinates(sample.jj[7].ref_x - 6, sample.jj[7].ref_y + 35)], 
                      4, 10, 52, 10, 22, 64, 1)

In [4]:
sample.loops[7].x2, sample.loops[7].y2

(10970.0, 3499)

In [3]:
sample = Sample(2)
sample.Generate_Resonators([coordinates(0, 0), coordinates(3000, 0), coordinates(7000, 0), coordinates(11000, 0),
                           coordinates(0, 3000), coordinates(3000, 3000), coordinates(7000, 3000), coordinates(11000, 3000),
                           coordinates(0, 6000), coordinates(3000, 6000), coordinates(7000, 6000), coordinates(11000, 6000)],
                           [125, 125, 125, 125, 125, 125, 175, 175, 175, 175, 175, 175], 
                           [5, 10, 15, 20, 25, 30, 5, 10, 15, 20, 25, 30])
#for i in range(len(sample.claw_center)):
    #sample.claw_center[i].x -= 4
    #sample.claw_center[i].y -= 2
sample.Generate_JJ([sample.claw_center[0], sample.claw_center[1], sample.claw_center[2], sample.claw_center[3],
                   sample.claw_center[4], sample.claw_center[5], sample.claw_center[6], sample.claw_center[7],
                   sample.claw_center[8], sample.claw_center[9], sample.claw_center[10], sample.claw_center[11]], 
                   [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3], 
                   [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
                   [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25], 
                   [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
                   [5, 10, 15, 20, 25, 30, 5, 10, 15, 20, 25, 30], 
                   [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7], 
                   [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], 9)
#sample.Generate_JJ([sample.resonators[1].claw_center], [0.25], [0.695], [0.1], [0.79], [2.2], [0.29], [2.2], [0.1], 1)
sample.Generate_Loops([coordinates(sample.jj[0].ref_x - 6, sample.jj[0].ref_y + 5), 
                       coordinates(sample.jj[1].ref_x - 6, sample.jj[1].ref_y + 10),
                       coordinates(sample.jj[2].ref_x - 6, sample.jj[2].ref_y + 15), 
                       coordinates(sample.jj[3].ref_x - 6, sample.jj[3].ref_y + 20),
                       coordinates(sample.jj[4].ref_x - 6, sample.jj[4].ref_y + 25), 
                       coordinates(sample.jj[5].ref_x - 6, sample.jj[5].ref_y + 30),
                       coordinates(sample.jj[6].ref_x - 6, sample.jj[6].ref_y + 5), 
                       coordinates(sample.jj[7].ref_x - 6, sample.jj[7].ref_y + 10),
                       coordinates(sample.jj[8].ref_x - 6, sample.jj[8].ref_y + 15), 
                       coordinates(sample.jj[9].ref_x - 6, sample.jj[9].ref_y + 20),
                       coordinates(sample.jj[10].ref_x - 6, sample.jj[10].ref_y + 25), 
                       coordinates(sample.jj[11].ref_x - 6, sample.jj[11].ref_y + 30)], 
                      4, 10, 52, 10, 22, 64, 1)


In [6]:
gdspy.LayoutViewer(depth = 2)

<gdspy.viewer.LayoutViewer object .!layoutviewer>

In [ ]:
z = [np.log(w)/w for w in np.arange(1,25,0.1)]
import matplotlib.pyplot as plt
plt.plot(np.arange(1,25,0.1), z)

In [4]:
gdspy.write_gds('capacitances_eq_1e-13.gds', unit=1.0e-6, precision=1.0e-9)

In [11]:
sample.jj[0].ref_x, sample.jj[1].ref_x, sample.jj[0].ref_y, sample.jj[1].ref_y

(2065.2, 3397.785628495315, 3851.4648470115208, 1485.0)

In [7]:
sample.resonators[0].claw_center

(2065.2, 3851.4648470115208)

In [8]:
sample.resonators[1].claw_center

(3405.785628495315, 1494.0)

In [ ]:
sample = Sample(1)
### А ты реально используешь х1, х2 для петелек?
sample.Generate_Pads(5000, 5000, 4, [17, 17, 10, 10], [33, 33, 22, 22], [75, 75, 64, 64], 0, 
                     [coordinates(800, 2500), coordinates(4200, 2500),
                     coordinates(2500, 800), coordinates(2500, 4200)])
sample.Generate_TL(0, 1, 17, 33, 75, 0)
sample.Generate_Resonators([coordinates(sample.tl.start.x + 600, 
                                        sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d),
                            coordinates(sample.tl.start.x + 1800, 
                                        sample.tl.start.y - (sample.tl.d2 - sample.tl.d)/2)], 
                           [sample.tl.start.x + 1450, sample.tl.end.x - 600], [8e9, 9e9], ['up', 'down'], [0, 0], True)
sample.resonators[0].claw_center.x -= 4
sample.resonators[1].claw_center.x -= 4#-8
sample.resonators[0].claw_center.y -= 3
sample.resonators[1].claw_center.y -= 9
sample.Generate_JJ([sample.resonators[0].claw_center, sample.resonators[1].claw_center], [0.25, 0.25], 
                   [0.695, 0.695], [0.1, 0.1], [0.79, 0.79], [2.2, 2.2], 
                   [0.29, 0.29], [2.2, 2.2], [0.1, 0.1], 9)
#sample.Generate_JJ([sample.resonators[1].claw_center], [0.25], [0.695], [0.1], [0.79], [2.2], [0.29], [2.2], [0.1], 1)
sample.Generate_Loops([coordinates(sample.jj[1].ref_x, sample.jj[1].ref_y), coordinates(sample.jj[0].ref_x, sample.jj[0].ref_y)],
                      [coordinates(sample.jj[1].ref_x + 7, sample.jj[1].ref_y + 2), coordinates(sample.jj[0].ref_x - 3, sample.jj[0].ref_y + 10)], 
                      ['up', 'down'], 4, 10, 52, 10, 22, 64,
                      [[coordinates(2526, 1000), coordinates(3354, 1083)], [coordinates(2087, 4165), coordinates(2527, 4264)]],
                      [[rotations(np.pi, np.pi/2), rotations(3*np.pi/2, 2*np.pi)],[rotations(np.pi, np.pi/2), rotations(3*np.pi/2, 2*np.pi)]], layer = 0)
#sample.Generate_Loops([coordinates(2065, 3861.5), coordinates(2065, 3861.5)],
                      #[coordinates(3397, 1487), coordinates(2064, 3861)], ['up', 'down'], 4, 10, 52, 10, 22, 64,
                      #[[coordinates(2526, 1000), coordinates(3354, 1083)], [coordinates(2087, 4165), coordinates(2527, 4264)]],
                      #[[rotations(np.pi, np.pi/2), rotations(3*np.pi/2, 2*np.pi)],[rotations(np.pi, np.pi/2), rotations(3*np.pi/2, 2*np.pi)]], layer = 0)
#ref1 = gdspy.CellReference(sample.Pads_cell, (0, 0))
#sample_cell = gdspy.Cell('Sample')
#sample_cell.add([ref1])

In [ ]:
loop = Loop(0, 1000, 0, 1000, 10, 22, 64, 17, 33, 75)
cell = gdspy.Cell('L2')
cell.add(loop.Generate())

In [2]:
cell = gdspy.Cell('Res1')
res = Resonator(0, 0, 1050, 8e9, 0)
cell.add(res.Generate(mode = 'up'))

In [5]:
sample.Generate_JJ([coordinates(2065, 3851)], [0.25], [0.695], [0.1], [0.79], [2.2], [0.29], [2.2], [0.1], 1)
sample.Generate_JJ([coordinates(3397, 1485)], [0.25], [0.695], [0.1], [0.79], [2.2], [0.29], [2.2], [0.1], 1)

In [6]:
j = JJ(0.25, 0.695, 0.1, 0.79, 2.2, 0.29, 2.2, 0.1, 1)
cell = gdspy.Cell('J3')
cell.add(j.Generate(2064, 3853))

In [5]:
cell = gdspy.Cell('Res')
res = Resonator(0, 0, 1050, 8e9, 0)
cell.add(res.Generate(mode = 'down'))

-28.0


In [3]:
res.Generate(mode = 'down').polygons

[array([[ -837.2 , -1093.71],
        [-1036.2 , -1093.71],
        [-1036.2 , -1109.71],
        [ -837.2 , -1109.71],
        [ -837.2 , -1137.71],
        [ -821.2 , -1137.71],
        [ -821.2 , -1109.71],
        [ -622.2 , -1109.71],
        [ -622.2 , -1093.71],
        [ -821.2 , -1093.71],
        [ -821.2 ,  -961.71],
        [ -837.2 ,  -961.71]]), array([[ -881.2 , -1038.96],
        [ -861.2 , -1038.96],
        [ -861.2 ,  -938.96],
        [ -797.2 ,  -938.96],
        [ -797.2 , -1038.96],
        [ -777.2 , -1038.96],
        [ -777.2 ,  -931.96],
        [ -881.2 ,  -931.96]]), array([[ -216.   ,   -75.   ],
        [ -982.   ,   -75.   ],
        [ -984.274,   -75.041],
        [ -986.546,   -75.164],
        [ -988.811,   -75.369],
        [ -991.068,   -75.656],
        [ -993.313,   -76.024],
        [ -995.543,   -76.473],
        [ -997.756,   -77.002],
        [ -999.947,   -77.611],
        [-1002.116,   -78.298],
        [-1004.258,   -79.063],
        [-1006

In [2]:
class JJ:
    def __init__(self, a, b, c, d, h, w1, w2, layer):
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        self.h = h 
        self.w1 = w1
        self.w2 = w2
        self.layer = layer 
        
    def Generate(self, x, y, mode = 'up'):
        #x = x + 4
        lower_part = gdspy.boolean(gdspy.Rectangle((x - self.w2/2, y - 2), (x + self.w2/2, y + 1)), 
                                   gdspy.boolean(gdspy.Rectangle((x - self.w2/2, y + 1), (x - self.w2/2 + self.c, y + int(self.h/2) - 0.16)), 
                                                 gdspy.Rectangle((x + self.w2/2, y + 1), (x + self.w2/2 - self.d, y + int(self.h/2) - 0.16)), 'or'), 'or')
        #lower_part = gdspy.boolean(gdspy.boolean(lower_part, 
                                                 #dspy.Rectangle((x + 2 - self.a, y + 5), (x + 2, y + 5 + self.b)), 'or'),
                                   #gdspy.Rectangle((x + 6 + self.c, y + 5), (x + 6, y + 5 + self.d)), 'or')
        upper_part = gdspy.boolean(gdspy.Rectangle((x - self.w1/2, y + self.h - 1), (x + self.w1/2, y + self.h + 2)), 
                                   gdspy.boolean(gdspy.Rectangle((x - self.w1/2, y + self.h - 1), (x - self.w1/2 + 0.5, y + int(self.h/2))), 
                                                 gdspy.Rectangle((x + self.w1/2, y + self.h - 1), (x + self.w1/2 - 0.5, y + int(self.h/2))), 'or'), 'or')
        upper_part = gdspy.boolean(gdspy.boolean(upper_part, 
                                                 gdspy.Rectangle((x - self.w1/2, y + int(self.h/2)), (x + 2.8 - self.w1/2, y + int(self.h/2) + self.a)), 'or'),
                                   gdspy.Rectangle((x + self.w1/2, y + int(self.h/2)), (x + self.w1/2 - 2.8, y + int(self.h/2) + self.b)), 'or')
        self.ref_x = x
        self.ref_y = y
        res = gdspy.boolean(upper_part, lower_part, 'or', layer = self.layer)
        if mode == 'down':
            res.rotate(-np.pi, (x, y))
            self.ref_y = y + h 
        return res



class Loop:
    
    def __init__(self, x1, x2, y1, y2, d, d1, d2, d_tl, d1_tl, d2_tl, layer):
        self.x1 = x1 
        self.x2 = x2 - 0.5*(d2 - d)
        self.y1 = y1
        self.y2 = y2
        self.d = d
        self.d1 = d1
        self.d2 = d2
        self.d_tl = d_tl
        self.d1_tl = d1_tl
        self.d2_tl = d2_tl
        self.layer = layer
        
    def Generate(self, mode = 'up'):
        end = gdspy.boolean(gdspy.boolean(gdspy.Rectangle((self.x2, self.y2), (self.x2 + self.d2, self.y2 - 150)),
                                          gdspy.Rectangle((self.x2 + 0.5*(self.d2 - self.d1), self.y2), (self.x2 + 0.5*(self.d2 + self.d1), self.y2 - 150)), 'not'),
                            gdspy.Rectangle((self.x2 + 0.5*(self.d2 - self.d), self.y2), (self.x2 + 0.5*(self.d2 + self.d), self.y2 - 150)), 'or')
        end = gdspy.boolean(end, gdspy.Rectangle((self.x2, self.y2), (self.x2 + 0.5*(self.d2 + self.d), self.y2 - 5.5)), 'or')
        end.rotate(np.pi, (self.x2 + 0.5*self.d2, self.y2))
        return end
        

In [ ]:
gdspy.LayoutViewer(depth = 2)

In [7]:
sample.tl.start.y, sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d

(2529.0, 2575.0)

In [ ]:
gdspy.LayoutViewer(depth = 2)

In [2]:
pads = Pads(5000, 5000, 4, [17, 17, 10, 10], [33, 33, 22, 22], [75, 75, 64, 64], 0,
                     [coordinates(800, 2500), coordinates(4200, 2500),
                     coordinates(2500, 800), coordinates(2500, 4200)])

In [5]:
sample = gdspy.Cell('Sample')
Pads_cell = gdspy.Cell('Pads_cell')
Pads_cell.add(pads.Generate_Ground())
ref1 = gdspy.CellReference(Pads_cell, (0, 0))
#sample.add([ref1])

ValueError: [GDSPY] Cell named Sample already present in library.

In [4]:
TL = Transmission_Line(pads, 0, 1, 17, 33, 75, 0)
transm = gdspy.Cell('TL')
transm.add(TL.Generate(coords = [], rots = [])) 
ref2 = gdspy.CellReference(transm, (0, 0))
sample.add([ref1, ref2])

NameError: name 'pads' is not defined

In [5]:
gdspy.write_gds('one_qubit.gds')

In [13]:
gdspy.Cell.add?

Object `gdspy.Cell.add` not found.


In [11]:
TL.start

NameError: name 'TL' is not defined

In [12]:
TL.end

NameError: name 'TL' is not defined

In [5]:
Resonator(0, 0, 650, 3750)

184.6105674500095 508.0


In [ ]:
sample.Generate_JJ([sample.claw_center[0], sample.claw_center[1], sample.claw_center[2], sample.claw_center[3],
                   sample.claw_center[4], sample.claw_center[5], sample.claw_center[6], sample.claw_center[7]], 
                   [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25], 
                   [0.695, 0.695, 0.695, 0.695, 0.695, 0.695, 0.695, 0.695], 
                   [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], 
                   [0.79, 0.79, 0.79, 0.79, 0.79, 0.79, 0.79, 0.79], 
                   [2.2, 2.2, 2.2, 2.2, 2.2, 2.2, 2.2, 2.2], 
                   [0.29, 0.29, 0.29, 0.29, 0.29, 0.29, 0.29, 0.29], 
                   [2.2, 2.2, 2.2, 2.2, 2.2, 2.2, 2.2, 2.2], 
                   [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], 9)



class JJ:
    def __init__(self, a, b, c, d, a1, b1, c1, d1, layer):
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        self.a1 = a1
        self.b1 = b1
        self.c1 = c1
        self.d1 = d1
        self.layer = layer 
        
    def Generate(self, x, y, mode = 'up'):
        #x = x + 4
        lower_part = gdspy.boolean(gdspy.Rectangle((x, y), (x + 8, y + 3)), 
                                   gdspy.boolean(gdspy.Rectangle((x + 1.35, y + 3), (x + 2, y + 5)), 
                                                 gdspy.Rectangle((x + 6, y + 3), (x + 6.65, y + 5)), 'or'), 'or')
        lower_part = gdspy.boolean(gdspy.boolean(lower_part, 
                                                 gdspy.Rectangle((x + 2 - self.a, y + 5), (x + 2, y + 5 + self.b)), 'or'),
                                   gdspy.Rectangle((x + 6 + self.c, y + 5), (x + 6, y + 5 + self.d)), 'or')
        upper_part = gdspy.boolean(gdspy.Rectangle((x, y + 12), (x + 8, y + 9)), 
                                   gdspy.boolean(gdspy.Rectangle((x + 0.25, y + 9), (x + 0.9, y + 6.145)), 
                                                 gdspy.Rectangle((x + 7.1, y + 9), (x + 7.45, y + 6.05)), 'or'), 'or')
        upper_part = gdspy.boolean(gdspy.boolean(upper_part, 
                                                 gdspy.Rectangle((x + 0.25, y + 6.145), (x + 0.9 + self.a1, y + 6.145 - self.b1)), 'or'),
                                   gdspy.Rectangle((x + 7.45, y + 6.05), (x + 7.45 - self.c1, y + 6.05 - self.d1)), 'or')
        self.ref_x = x
        self.ref_y = y
        res = gdspy.boolean(upper_part, lower_part, 'or', layer = self.layer)
        if mode == 'down':
            res.rotate(-np.pi, (x + 4, y + 6))
            self.ref_y = y + 9
        return res